# Week 4 Tasks
Connect your Pi Pico to your computer via Jupyter.

## Task 1:
Turn LED light on.
### Hardware Requriment
 1. Breadbord
 2. LED light
 3. Pi Pico
 4. Jumper wires

## Task 2:
Make LED light Blink at half second intervals.
### Hardware Requriment
 1. Breadbord
 2. LED light
 3. Pi Pico
 4. Jumper wires 

## Task 3:
Add two buttons.  Use one to turn the internal led on and the other to turn it off.
### Hardware Requriment
 1. Breadbord
 2. LED light
 3. Button
 4. Pi Pico
 5. Jumper wires 

## Task 4: Trafic light Prototype
### Scenario
A small municipality has hired you to prototype a simple traffic light controller for a quiet residential intersection. The local council wants an easy-to-follow Red-Yellow-Green cycle to regulate car traffic. Your job is to demonstrate a working model of the traffic light using a Raspberry Pi Pico (or any microcontroller) and three LEDs.

### Key Requirements
##### Traffic Light Cycle
 1. Red → Yellow → Green
 2. Red ON (enough time to ensure vehicles fully stop).
 3. Yellow ON (brief warning).
 4. Green ON (allow traffic flow).
 5. Cycle repeats continuously, with no overlap of multiple lights.
### Hardware Requriment
 1. Breadbord
 2. Three LED light (Green, Yellow, Red)
 3. Pi Pico
 4. Jumper wires 


## Task 5: Basic Traffic light with Pedestrian Button and light
### Scenario
After successfully demonstrating a basic traffic light prototype, the city council of “Greenfield Heights” has requested an enhanced version to accommodate pedestrians at a busy section of Main Street safely. 
Specifically, they want:

1.	Retain the existing traffic light cycle (Red, Yellow, Green) for vehicles.
2.	Add a pedestrian signal with Green (Walk) and Red (Don’t Walk) lights.
3.	Include a pedestrian button so pedestrians can request to cross.
### Hardware Requirement
 1. Breadbord
 2. Five LED lights (Green, Yellow, Red, for vehicles. Green and Red for Predistrain)
 3. Button
 4. Pi Pico
 5. Jumper wires 
